<a href="https://colab.research.google.com/github/pswami12/Temporay_add_to_version_control/blob/master/Case_Study_2_(Cyber_Sec)_P_Swami_Sachidananda_FullStackAppliedAI_Aug22WeekendBatch_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Business Objective**

As part of this project, your task is to build network intrusion detection system to detect anomalies and attacks in the network. There are two problems: 
i.  Binomial classification: Detect anomalies by predicting Activity is normal or attack 
ii. Multinomial Classification: Detecting type of activity by predicting Activity is Normal or Back or Buffer Over         flow or FTP Write or Guess Password or Neptune or N-Map or Port Sweep or Root Kit or Satan or Smurf

You are required to append all the files and create new column called attack based on the name of attack. While you are appending the files, you can take resampling of data based on the number of attacks. 

For Binomial classification, you can create attack variable with attack vs. normal 

For Multinomial classification, you can create attack variable with normal vs. Back vs. Buffer Over flow vs. FTP Write vs. Guess Password vs. Neptune vs. N-Map vs. Port Sweep vs. Root Kit vs. Satan vs. Smurf

### **Importing Packages for building classification models using Machine Learning**

In [4]:
#Packages related to general operating system & warnings
import os 
import warnings
warnings.filterwarnings('ignore')

#Packages related to data importing, manipulation, exploratory data analysis, data understanding
import numpy as np
import pandas as pd
import scipy.stats as stats

#Packages related to data visualizaiton
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

#Setting plot sizes and type of plot
plt.rc("font", size=14)
plt.rcParams['axes.grid'] = True
plt.figure(figsize=(6,3))
plt.gray()

from matplotlib.backends.backend_pdf import PdfPages

#Modules related to split the data & gridsearch
from sklearn.model_selection import train_test_split, GridSearchCV

#Module related to calculation of metrics
from sklearn import metrics

#Module related to VIF 
from statsmodels.stats.outliers_influence import variance_inflation_factor

#Modules related to preprocessing (Imputation of missings, standardiszation, new features creation, converting categorical to numerical)
from sklearn.impute import MissingIndicator, SimpleImputer
from sklearn.preprocessing import  PolynomialFeatures, KBinsDiscretizer, FunctionTransformer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, LabelBinarizer, OrdinalEncoder

#Moudles related to feature selection
from sklearn.feature_selection import RFE, RFECV, SelectKBest, chi2, SelectPercentile, f_classif, mutual_info_classif, f_regression, VarianceThreshold, SelectFromModel, mutual_info_classif, mutual_info_regression, SelectFpr, SelectFdr, SelectFwe

#Modules related to pipe line creation for faster processing
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion

#Modules related key techniques of supervised learning 
import statsmodels.formula.api as smf
import statsmodels.tsa as tsa

from sklearn.linear_model import LogisticRegression, LinearRegression, ElasticNet, Lasso, Ridge
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, export_graphviz
from sklearn.ensemble import BaggingClassifier, BaggingRegressor,RandomForestClassifier,RandomForestRegressor
from sklearn.ensemble import GradientBoostingClassifier,GradientBoostingRegressor, AdaBoostClassifier, AdaBoostRegressor 
from sklearn.naive_bayes import BernoulliNB, MultinomialNB, GaussianNB
from sklearn.svm import LinearSVC, LinearSVR, SVC, SVR

<Figure size 600x300 with 0 Axes>

In [5]:
# List of table names
table_names = [
    'Data_of_Attack_Back_Normal', 'Data_of_Attack_Back', 'Data_of_Attack_Back_BufferOverflow',
    'Data_of_Attack_Back_FTPWrite', 'Data_of_Attack_Back_GuessPassword', 'Data_of_Attack_Back_Neptune',
    'Data_of_Attack_Back_NMap', 'Data_of_Attack_Back_PortSweep', 'Data_of_Attack_Back_RootKit',
    'Data_of_Attack_Back_Satan', 'Data_of_Attack_Back_Smurf'
]

# Create an empty DataFrame to store the appended tables
final_data = pd.DataFrame()

# Iterate over the table names and append the tables to the DataFrame
for table_name in table_names:
    # Read the table data
    table_data = pd.read_csv(f'{table_name}.csv')  
    
    # Extract the attack type from the table name
    attack_type = table_name.replace('Data_of_Attack_', '')
    
    # Add the 'Attack' field with the attack type as a value
    table_data['Attack'] = attack_type
    
    # Append the table data to the final_data DataFrame
    final_data = final_data.append(table_data, ignore_index=True)

final_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 817551 entries, 0 to 817550
Data columns (total 42 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   duration                      817551 non-null  float64
 1    protocol_type                817551 non-null  float64
 2    service                      817551 non-null  float64
 3    flag                         817551 non-null  float64
 4    src_bytes                    817551 non-null  float64
 5    dst_bytes                    817551 non-null  float64
 6    land                         817551 non-null  int64  
 7    wrong_fragment               817551 non-null  float64
 8    urgent                       817551 non-null  float64
 9    hot                          817551 non-null  float64
 10   num_failed_logins            817551 non-null  float64
 11   logged_in                    817551 non-null  float64
 12   num_compromised              817551 non-nul

In [6]:
final_data ['Attack'].value_counts()

Back_Normal            576710
Back_Neptune           227228
Back_Satan               5019
Back_Smurf               3007
Back_PortSweep           2964
Back_NMap                1554
Back                      968
Back_GuessPassword         53
Back_BufferOverflow        30
Back_RootKit               10
Back_FTPWrite               8
Name: Attack, dtype: int64

In [7]:
final_data.isna().sum()

duration                        0
 protocol_type                  0
 service                        0
 flag                           0
 src_bytes                      0
 dst_bytes                      0
 land                           0
 wrong_fragment                 0
 urgent                         0
 hot                            0
 num_failed_logins              0
 logged_in                      0
 num_compromised                0
 root_shell                     0
 su_attempted                   0
 num_root                       0
 num_file_creations             0
 num_shells                     0
 num_access_files               0
 num_outbound_cmds              0
 is_host_login                  0
 is_guest_login                 0
 count                          0
 srv_count                      0
 serror_rate                    0
 srv_error_rate                 0
 rerror_rate                    0
 srv_rerror_rate                0
 same_srv_rate                  0
 diff_srv_rate

**Using SMOTE to handle imbalanced data**

In [8]:
from imblearn.over_sampling import SMOTE

X = final_data.drop('Attack', axis=1)
y = final_data['Attack']

# Initialize SMOTE for oversampling
smote = SMOTE()

# Perform SMOTE resampling
X_resampled, y_resampled = smote.fit_resample(X, y)

# Create a new balanced DataFrame
final_data_balanced = pd.DataFrame(X_resampled, columns=X.columns)
final_data_balanced['Attack'] = y_resampled


final_data_balanced.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6343810 entries, 0 to 6343809
Data columns (total 42 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   duration                      float64
 1    protocol_type                float64
 2    service                      float64
 3    flag                         float64
 4    src_bytes                    float64
 5    dst_bytes                    float64
 6    land                         int64  
 7    wrong_fragment               float64
 8    urgent                       float64
 9    hot                          float64
 10   num_failed_logins            float64
 11   logged_in                    float64
 12   num_compromised              float64
 13   root_shell                   float64
 14   su_attempted                 float64
 15   num_root                     float64
 16   num_file_creations           float64
 17   num_shells                   float64
 18   num_access_files     

In [9]:
final_data_balanced.isna().sum()

duration                        0
 protocol_type                  0
 service                        0
 flag                           0
 src_bytes                      0
 dst_bytes                      0
 land                           0
 wrong_fragment                 0
 urgent                         0
 hot                            0
 num_failed_logins              0
 logged_in                      0
 num_compromised                0
 root_shell                     0
 su_attempted                   0
 num_root                       0
 num_file_creations             0
 num_shells                     0
 num_access_files               0
 num_outbound_cmds              0
 is_host_login                  0
 is_guest_login                 0
 count                          0
 srv_count                      0
 serror_rate                    0
 srv_error_rate                 0
 rerror_rate                    0
 srv_rerror_rate                0
 same_srv_rate                  0
 diff_srv_rate

In [10]:
final_data_balanced['Attack'].value_counts()

Back_Normal            576710
Back                   576710
Back_BufferOverflow    576710
Back_FTPWrite          576710
Back_GuessPassword     576710
Back_Neptune           576710
Back_NMap              576710
Back_PortSweep         576710
Back_RootKit           576710
Back_Satan             576710
Back_Smurf             576710
Name: Attack, dtype: int64

In [11]:
final_data_balanced.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6343810 entries, 0 to 6343809
Data columns (total 42 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   duration                      float64
 1    protocol_type                float64
 2    service                      float64
 3    flag                         float64
 4    src_bytes                    float64
 5    dst_bytes                    float64
 6    land                         int64  
 7    wrong_fragment               float64
 8    urgent                       float64
 9    hot                          float64
 10   num_failed_logins            float64
 11   logged_in                    float64
 12   num_compromised              float64
 13   root_shell                   float64
 14   su_attempted                 float64
 15   num_root                     float64
 16   num_file_creations           float64
 17   num_shells                   float64
 18   num_access_files     

**Taking sample from above data**

In [12]:
# Calculate the sample size (20% of the total data)
sample_size = int(0.2 * len(final_data_balanced))

# Take a random sample of 20% of the data
final_data_balanced_sample = final_data_balanced.sample(n=sample_size, random_state=42)

# Display the sample size
print(sample_size)
final_data_balanced_sample

1268762


,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,Attack
4102946,0.913358,0.000000,0.100000,0.080000,0.000010,0.000000,0,0.0,0.00000,0.00000,...,0.002000,0.001000,0.074000,0.100000,0.000,0.000,0.0,0.100000,0.100000,Back_PortSweep
5161978,0.022049,0.000000,0.050000,0.000000,0.008431,0.115133,0,0.0,0.07992,0.07992,...,0.003799,0.001799,0.002000,0.000000,0.000,0.000,0.0,0.000000,0.000000,Back_RootKit
1546620,0.020328,0.000000,0.083006,0.037996,0.009537,0.030360,0,0.0,0.00000,0.00000,...,0.004900,0.100000,0.000000,0.047438,0.000,0.000,0.0,0.010766,0.010766,Back_BufferOverflow
1289141,0.000000,0.000000,0.000000,0.000000,0.545400,0.083140,0,0.0,0.00000,0.20000,...,0.138522,0.100000,0.000000,0.001000,0.000,0.000,0.0,0.001000,0.001000,Back
5394503,0.000000,0.000000,0.120000,0.050000,0.000000,0.000000,0,0.0,0.00000,0.00000,...,0.001000,0.000000,0.084838,0.000000,0.000,0.007,0.0,0.062838,0.100000,Back_Satan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4850854,0.031124,0.005604,0.078020,0.000000,0.007771,0.105856,0,0.0,0.00000,0.00000,...,0.001879,0.000440,0.002000,0.000000,0.000,0.000,0.0,0.000000,0.000000,Back_RootKit
973033,0.000000,0.000000,0.000000,0.000000,0.545400,0.083140,0,0.0,0.00000,0.20000,...,0.255000,0.100000,0.000000,0.000000,0.000,0.000,0.0,0.003556,0.003556,Back
3239729,0.000000,0.010000,0.120000,0.000000,0.001000,0.000893,0,0.0,0.00000,0.00000,...,0.039893,0.017000,0.001000,0.017000,0.000,0.000,0.0,0.000000,0.000000,Back_NMap
3854298,0.000000,0.000000,0.430000,0.070000,0.000000,0.000000,0,0.0,0.00000,0.00000,...,0.001000,0.000000,0.006012,0.000000,0.000,0.100,0.1,0.000000,0.000000,Back_Neptune


In [13]:
final_data_balanced_sample['Attack'].value_counts()

Back_FTPWrite          115780
Back_RootKit           115582
Back_Smurf             115547
Back_Normal            115537
Back_Neptune           115481
Back_GuessPassword     115282
Back_Satan             115220
Back_NMap              115214
Back_BufferOverflow    115090
Back                   115024
Back_PortSweep         115005
Name: Attack, dtype: int64

**Saving the sample in google colab**

In [14]:
# Specify the path where you want to save the CSV file in Google Colab
colab_csv_path = '/content/final_data_balanced_sample.csv'

# Export the sample_data DataFrame to a CSV file in Google Colab
final_data_balanced_sample.to_csv(colab_csv_path, index=False)

In [15]:
from google.colab import drive
drive.mount('/content/drive')
# Assuming sample_data is your DataFrame containing the sample

# Specify the path where you want to save the CSV file in Google Drive
drive_csv_path = '/content/drive/MyDrive/final_data_balanced_sample.csv'

# Export the sample_data DataFrame to a CSV file in Google Drive
final_data_balanced_sample.to_csv(drive_csv_path, index=False)


import shutil

# Specify the path where you want to save the CSV file on your MacBook's desktop
local_csv_path = '/content/final_data_balanced_sample.csv'

# Copy the CSV file from Google Drive to Colab's local file system
shutil.copy(drive_csv_path, local_csv_path)

Mounted at /content/drive


'/content/final_data_balanced_sample.csv'

In [18]:
# Download the CSV file from Colab to your MacBook's desktop
from google.colab import files
files.download(local_csv_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>